In [13]:
import os
import json
import PyPDF2
from qdrant_client import QdrantClient
from qdrant_client.http.models import PointStruct, VectorParams
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_qdrant import QdrantVectorStore
from langchain_aws import BedrockLLM
from langchain.chains import RetrievalQA

In [14]:

# Parámetros Qdrant
QDRANT_HOST = os.getenv("QDRANT_HOST", "localhost")
QDRANT_PORT = int(os.getenv("QDRANT_PORT", 6333))
COLLECTION_NAME = "documentos"

# Parámetros Amazon Bedrock
# Si usas perfil AWS:
BEDROCK_PROFILE = os.getenv("BEDROCK_PROFILE", "bedrock-admin")
# Modelo a usar en Bedrock
BEDROCK_MODEL_ID = os.getenv("BEDROCK_MODEL_ID", "amazon.titan-text-express-v1")


In [15]:
# Función para leer PDF completo a texto
def read_pdf(file_path: str) -> str:
    text = ""
    with open(file_path, 'rb') as f:
        reader = PyPDF2.PdfReader(f)
        for page in reader.pages:
            if (p := page.extract_text()):
                text += p + "\n"
    return text

In [16]:
def chunk_text(text: str, chunk_size: int = 1000, overlap: int = 100) -> list[str]:
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start += chunk_size - overlap
    return chunks

In [17]:
# 3.1 Inicializar clientes y modelos
qdrant_client = QdrantClient(host=QDRANT_HOST, port=QDRANT_PORT)
embedder = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# 3.2 Leer y chunkear PDF
pdf_path = "documento.pdf"
print("Leyendo PDF…")
document_text = read_pdf(pdf_path)
print("Dividiendo en chunks…")
chunks = chunk_text(document_text, chunk_size=1000, overlap=100)
print(f"{len(chunks)} chunks generados.")

# 3.3 Crear colección si no existe
try:
    # Intenta usar directamente la API de SentenceTransformer
    dimension = embedder.client.get_sentence_embedding_dimension()
except AttributeError:
    # En caso de que no exista, calcula la dimensión haciendo un embed de prueba
    dimension = len(embedder.embed_query("test"))
    print("Hola")
collections = qdrant_client.get_collections().collections
if not any(col.name == COLLECTION_NAME for col in collections):
    print("Creando colección en Qdrant…")
    qdrant_client.create_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=VectorParams(size=dimension, distance="Cosine")
    )
    print("Colección creada.")

# 3.4 Calcular embeddings y upsert
print("Calculando embeddings…")
vectors = embedder.embed_documents(chunks)
points = [PointStruct(id=i, vector=v, payload={"text": c})
          for i, (c, v) in enumerate(zip(chunks, vectors))]
print("Subiendo embeddings a Qdrant…")
qdrant_client.upsert(collection_name=COLLECTION_NAME, points=points)
print("Embeddings cargados correctamente.")

Leyendo PDF…
Dividiendo en chunks…
478 chunks generados.
Hola
Calculando embeddings…
Subiendo embeddings a Qdrant…
Embeddings cargados correctamente.


In [18]:
# 4.1 Crear vectorstore y retriever
tlr_vectorstore = QdrantVectorStore(
    client=qdrant_client,
    collection_name=COLLECTION_NAME,
    embedding=embedder
)
retriever = tlr_vectorstore.as_retriever(search_kwargs={"k": 5})

# 4.2 Inicializar LLM de Bedrock
# Opción A: con perfil AWS
tlr_llm = BedrockLLM(
    credentials_profile_name=BEDROCK_PROFILE,
    model_id=BEDROCK_MODEL_ID
)

# 4.3 Definir cadena RetrievalQA
tlr_qa_agent = RetrievalQA.from_chain_type(
    llm=tlr_llm,
    chain_type="refine",  # o map_reduce/refine según tus necesidades
    retriever=retriever,
    return_source_documents=False
)

In [19]:
# Función para procesar y guardar respuestas
def process_questions(input_json: str, output_json: str) -> None:
    with open(input_json, 'r', encoding='utf-8') as f:
        qa_list = json.load(f)

    results = []
    for item in qa_list:
        q = item.get("Q") or item.get("question")
        exp = item.get("A") or item.get("expected_answer")
        
        print(f"Evaluando pregunta: {q}")
        
        out = tlr_qa_agent.invoke({"query": q})
        gen = out.get("result", "")
        results.append({
            "question": q,
            "expected_answer": exp,
            "generated_answer": gen.strip() if isinstance(gen, str) else gen
        })

    with open(output_json, 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=4)
    print(f"Guardado: {output_json}")

# Ejecutar procesamiento para ambos conjuntos de preguntas
if __name__ == "__main__":
    process_questions("Expert-questions.json", "Expert-questions-output.json")
    process_questions("Not-expert-questions.json", "Not-expert-questions-output.json")

Evaluando pregunta: ¿Cuáles son los principales componentes metodológicos para identificar y medir los riesgos ESG?
Evaluando pregunta: ¿Cómo debe realizarse la evaluación de materialidad en el contexto ESG?
Evaluando pregunta: ¿Qué criterios se utilizan para determinar la materialidad de un riesgo ESG?
Evaluando pregunta: ¿Cómo se integran los riesgos ESG en los procesos de ICAAP e ILAAP?
Evaluando pregunta: ¿Qué papel juegan los análisis de escenarios en la gestión de riesgos ESG?
Evaluando pregunta: ¿Cuáles son los desafíos en la obtención y calidad de datos para evaluar riesgos ESG?
Evaluando pregunta: ¿Qué implicaciones tiene la integración de riesgos ESG en el apetito de riesgo de una institución?
Evaluando pregunta: ¿Cómo se aplican las metodologías de alineación de cartera en el análisis ESG?
Evaluando pregunta: ¿Qué importancia tienen las evaluaciones de vulnerabilidad a riesgos físicos?
Evaluando pregunta: ¿De qué forma se integran los riesgos de transición en los modelos de 